In [122]:
import sys
print(sys.executable)


d:\prediction_price_house_bpn\env\Scripts\python.exe


In [123]:
from sklearn.model_selection import train_test_split,  GridSearchCV
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [124]:
pd.set_option("display.max_rows", None)

# Baca CSV (hasil cleaning / dataset final)
df = pd.read_csv('data_final_bersih.csv')  # ganti sesuai nama file CSV-mu

# Tampilkan seluruh data
df

,Luas Tanah (m²),Luas Bangunan (m²),Kamar Tidur,Kamar Mandi,Daerah_Balikpapan Barat,Daerah_Balikpapan Kota,Daerah_Balikpapan Selatan,Daerah_Balikpapan Tengah,Daerah_Balikpapan Timur,Daerah_Balikpapan Utara,Harga (IDR)
0,250,250,1,1,0,0,1,0,0,0,1.120000e+09
1,90,60,2,1,0,0,0,1,0,0,9.250000e+08
2,100,68,2,2,0,0,1,0,0,0,5.750000e+08
3,92,72,3,2,0,0,0,1,0,0,6.957000e+08
4,144,280,3,2,0,0,1,0,0,0,1.650000e+09
5,213,90,3,1,0,0,1,0,0,0,7.950000e+08
6,120,145,4,2,0,0,1,0,0,0,1.250000e+09
7,120,250,3,2,0,0,1,0,0,0,1.400000e+09
8,165,220,4,2,0,0,0,0,0,1,1.650000e+09
9,180,320,4,4,0,0,0,0,0,1,1.850000e+09


In [125]:
# X = fitur (semua kecuali target)
X = df.drop(columns=['Harga (IDR)'])

# y = target
y = df['Harga (IDR)']

# Cek beberapa baris
print("--- Fitur (X) ---")
print(X.head())
print("\n--- Target (y) ---")
print(y.head())


--- Fitur (X) ---
   Luas Tanah (m²)  Luas Bangunan (m²)  Kamar Tidur  Kamar Mandi  \
0              250                 250            1            1   
1               90                  60            2            1   
2              100                  68            2            2   
3               92                  72            3            2   
4              144                 280            3            2   

   Daerah_Balikpapan Barat  Daerah_Balikpapan Kota  Daerah_Balikpapan Selatan  \
0                        0                       0                          1   
1                        0                       0                          0   
2                        0                       0                          1   
3                        0                       0                          0   
4                        0                       0                          1   

   Daerah_Balikpapan Tengah  Daerah_Balikpapan Timur  Daerah_Balikpapan Utara  
0     

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

(247, 62, 247, 62)

In [127]:
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.5069615583652887

In [128]:
param_grid = {
    'fit_intercept': [True, False],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best model
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)
best_model.score(X_test, y_test)

0.5069615583652887

In [129]:
# # Evaluasi
# r2 = model.score(X_test, y_test)
# print("R^2 Score:", r2)


In [130]:
# Simpan model ke file
pickle.dump(model, open('model.pkl', 'wb'))

In [131]:
import pickle
import numpy as np

# Load model
model = pickle.load(open('model.pkl', 'rb'))


In [132]:
# X_train, y_train sudah dibuat sebelumnya
residual = y_train - model.predict(X_train)
std_residual = np.std(residual)  # Standar deviasi residual

In [133]:
kecamatan_list = ['Balikpapan Barat', 'Balikpapan Kota', 'Balikpapan Selatan', 
             'Balikpapan Tengah', 'Balikpapan Timur', 'Balikpapan Utara']

In [134]:
def prediksi_harga(kamar_tidur, kamar_mandi, luas_tanah, luas_bangunan, kecamatan):
    # Encode kecamatan ke one-hot
    daerah = [0] * len(kecamatan_list)
    if kecamatan in kecamatan_list:
        daerah[kecamatan_list.index(kecamatan)] = 1
    else:
        raise ValueError("Kecamatan tidak valid!")

    # Susun fitur sesuai urutan X_train
    fitur = np.array([[luas_tanah, luas_bangunan, kamar_tidur, kamar_mandi] + daerah])
    
    # Prediksi harga
    prediksi = model.predict(fitur)[0]
    
    # Hitung range ±1 std residual
    bawah = prediksi - std_residual
    atas = prediksi + std_residual
    
    return prediksi, bawah, atas

In [135]:
harga, bawah, atas = prediksi_harga(
    kamar_tidur=4,
    kamar_mandi=2,
    luas_tanah=350,
    luas_bangunan=250,
    kecamatan='Balikpapan Selatan'
)

print(f"Prediksi harga: Rp{harga:,.0f}")
print(f"Range prediksi: Rp{bawah:,.0f} - Rp{atas:,.0f}")


Prediksi harga: Rp2,652,877,912
Range prediksi: Rp2,041,958,427 - Rp3,263,797,397


d:\prediction_price_house_bpn\env\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
